In [7]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [8]:
import pandas as pd


path = "/content/drive/My Drive/fire_predict/base_data/"
df_train = pd.read_csv(path + "PJT002_train.csv")
df_val = pd.read_csv(path + "PJT002_validation.csv")
df_test = pd.read_csv(path + "PJT002_test.csv")

df_eg_year = pd.read_csv(path + "df_eg_year.csv")
df_eg_year_val = pd.read_csv(path + "df_eg_year_val.csv")
df_eg_year_test = pd.read_csv(path + "df_eg_year_test.csv")

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (8,172,173,174,175,176,177,178,179) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
import math
import numpy as np
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from tqdm import tqdm


def f_data(train, val, test):

    X_train = train.drop(['fr_yn'], 1)
    y_train = train['fr_yn']
    X_val = val.drop(['fr_yn'], 1)
    y_val = val['fr_yn']
    X_test = test.drop(['fr_yn'], 1)

    df_all = pd.concat([X_train, X_val, X_test])

    categorical_cols = df_all.select_dtypes(['object']).columns
    for col in categorical_cols:
        df_all[col] = pd.Categorical(df_all[col]).codes

    X_train = df_all[:len(train)]
    X_val = df_all[len(train):-len(test)]
    X_test = df_all[-len(test):]

    X_train = X_train.fillna(-1)
    X_val = X_val.fillna(-1)
    X_test = X_test.fillna(-1)

    # X_train = X_train.replace({-1:np.NaN})
    # X_val = X_val.replace({-1:np.NaN})
    # X_test = X_test.replace({-1:np.NaN})

    return X_train, y_train, X_val, y_val, X_test

def f1_rfc(X_train, y_train, X_val, y_val):
    result_list = []
    for i in tqdm(range(10)):

        model = RandomForestClassifier(bootstrap=True, max_depth=80,
                                       max_features="auto", min_samples_leaf=4,
                                       min_samples_split=5, n_estimators=200,
                                       random_state=42)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_val)
        result = f1_score(y_val, y_pred)
        result_list.append(result)
    return sum(result_list) / len(result_list)

def f1_rfc_2(X_train, y_train, X_val, y_val):
    result_list = []
    for i in tqdm(range(10)):

        model = RandomForestClassifier(random_state=42, n_estimators=1000)
        model.fit(X_train, y_train)

        y_pred = model.predict(X_val)
        result = f1_score(y_val, y_pred)
        result_list.append(result)
    return sum(result_list) / len(result_list)

# lable -> binary
binary_y = {'N': 0, 'Y': 1}

df_train['fr_yn'] = df_train['fr_yn'].map(binary_y)
df_val['fr_yn'] = df_val['fr_yn'].map(binary_y)

#df ->df_2
df_train_2 = df_train
df_val_2 = df_val
df_test_2 = df_test

#drop id
df_train_2 = df_train_2.drop("id", axis=1)
df_val_2 = df_val_2.drop("id", axis=1)
df_test_2 = df_test_2.drop("id", axis=1)

#drop dt_of_fr
df_train_2 = df_train_2.drop("dt_of_fr", axis=1)
df_val_2 = df_val_2.drop("dt_of_fr", axis=1)
df_test_2 = df_test_2.drop("dt_of_fr", axis=1)

#dt_of_athrztn -> year 
def year(x):
    if x != "NaN":
        if isinstance(x, float):
            if not math.isnan(x):
                if int(x) >10000000:
                    return x // 10000
                elif int(x) > 1000000:
                    return x // 1000
                elif int(x) > 100000:
                    return x // 100
                elif int(x) > 10000:
                    return x // 10
                elif int(x) > 1000:
                    return x
        elif isinstance(x, str):
            return int(x[:4])

df_train_2["dt_of_athrztn"] = df_train_2["dt_of_athrztn"].apply(year)
df_train_2["dt_of_athrztn"] = df_train_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)
df_val_2["dt_of_athrztn"] = df_val_2["dt_of_athrztn"].apply(year)
df_val_2["dt_of_athrztn"] = df_val_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)
df_test_2["dt_of_athrztn"] = df_test_2["dt_of_athrztn"].apply(year)
df_test_2["dt_of_athrztn"] = df_test_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)

# hm_cnt -> categorical
# def population(x):
#     if x < 1000:
#         return "1k"
#     elif x < 10000:
#         return "10k"
#     elif x < 100000:
#         return "100k"
#     elif x < 1000000:
#         return "1000k"

# df_train_2["hm_cnt"] = df_train_2["hm_cnt"].apply(population)
# df_val_2["hm_cnt"] = df_val_2["hm_cnt"].apply(population)
# df_test_2["hm_cnt"] = df_test_2["hm_cnt"].apply(population)

#tmprtr c -> k
df_train_2.tmprtr = df_train_2.tmprtr.apply(lambda x: x + 273.15 if True else x)
df_val_2.tmprtr = df_val_2.tmprtr.apply(lambda x: x + 273.15 if True else x)
df_test_2.tmprtr = df_test_2.tmprtr.apply(lambda x: x + 273.15 if True else x)

#gas_engry_us_201507 x < 0 -> remove
df_train_2.gas_engry_us_201507 = df_train_2.gas_engry_us_201507.apply(lambda x: None if x < 0 else x)

# drop gas, elec
gas_elec_columns = list(df_train_2.iloc[:,16:136].columns)
df_train_2 = df_train_2.drop(gas_elec_columns, axis=1)
df_val_2 = df_val_2.drop(gas_elec_columns, axis=1)
df_test_2 = df_test_2.drop(gas_elec_columns, axis=1)

#concat eg_year
df_train_2 = pd.concat([df_train_2, df_eg_year], axis=1)
df_val_2 = pd.concat([df_val_2, df_eg_year_val], axis=1)
df_test_2 = pd.concat([df_test_2, df_eg_year_test], axis=1)

#for imputation
# s = df_train_2.isnull().sum()
# c_list = np.where(s >= max(s)*0.5)
# df_train_2 = df_train_2.drop(list(s.index[c_list[0]]), axis=1)

# df_val_2 = df_val_2.drop(list(s.index[c_list[0]]), axis=1)
# df_test_2 = df_test_2.drop(list(s.index[c_list[0]]), axis=1)

X_train, y_train, X_val, y_val, X_test = f_data(df_train_2, df_val_2, df_test_2)

In [0]:
def preprocessing():
    path = "/content/drive/My Drive/fire_predict/base_data/"
    df_train = pd.read_csv(path + "PJT002_train.csv")
    df_val = pd.read_csv(path + "PJT002_validation.csv")
    df_test = pd.read_csv(path + "PJT002_test.csv")

    df_eg_year = pd.read_csv(path + "df_eg_year.csv")
    df_eg_year_val = pd.read_csv(path + "df_eg_year_val.csv")
    df_eg_year_test = pd.read_csv(path + "df_eg_year_test.csv")

    # lable -> binary
    binary_y = {'N': 0, 'Y': 1}

    df_train['fr_yn'] = df_train['fr_yn'].map(binary_y)
    df_val['fr_yn'] = df_val['fr_yn'].map(binary_y)

    #df ->df_2
    df_train_2 = df_train
    df_val_2 = df_val
    df_test_2 = df_test

    #drop id
    df_train_2 = df_train_2.drop("id", axis=1)
    df_val_2 = df_val_2.drop("id", axis=1)
    df_test_2 = df_test_2.drop("id", axis=1)

    #drop dt_of_fr
    df_train_2 = df_train_2.drop("dt_of_fr", axis=1)
    df_val_2 = df_val_2.drop("dt_of_fr", axis=1)
    df_test_2 = df_test_2.drop("dt_of_fr", axis=1)

    #dt_of_athrztn -> year 

    df_train_2["dt_of_athrztn"] = df_train_2["dt_of_athrztn"].apply(year)
    df_train_2["dt_of_athrztn"] = df_train_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)
    df_val_2["dt_of_athrztn"] = df_val_2["dt_of_athrztn"].apply(year)
    df_val_2["dt_of_athrztn"] = df_val_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)
    df_test_2["dt_of_athrztn"] = df_test_2["dt_of_athrztn"].apply(year)
    df_test_2["dt_of_athrztn"] = df_test_2["dt_of_athrztn"].apply(lambda x: None if x > 3000 else x)

    # hm_cnt -> categorical
    # def population(x):
    #     if x < 1000:
    #         return "1k"
    #     elif x < 10000:
    #         return "10k"
    #     elif x < 100000:
    #         return "100k"
    #     elif x < 1000000:
    #         return "1000k"

    # df_train_2["hm_cnt"] = df_train_2["hm_cnt"].apply(population)
    # df_val_2["hm_cnt"] = df_val_2["hm_cnt"].apply(population)
    # df_test_2["hm_cnt"] = df_test_2["hm_cnt"].apply(population)

    #tmprtr c -> k
    df_train_2.tmprtr = df_train_2.tmprtr.apply(lambda x: x + 273.15 if True else x)
    df_val_2.tmprtr = df_val_2.tmprtr.apply(lambda x: x + 273.15 if True else x)
    df_test_2.tmprtr = df_test_2.tmprtr.apply(lambda x: x + 273.15 if True else x)

    #gas_engry_us_201507 x < 0 -> remove
    df_train_2.gas_engry_us_201507 = df_train_2.gas_engry_us_201507.apply(lambda x: None if x < 0 else x)

    # drop gas, elec
    gas_elec_columns = list(df_train_2.iloc[:,16:136].columns)
    df_train_2 = df_train_2.drop(gas_elec_columns, axis=1)
    df_val_2 = df_val_2.drop(gas_elec_columns, axis=1)
    df_test_2 = df_test_2.drop(gas_elec_columns, axis=1)

    #concat eg_year
    df_train_2 = pd.concat([df_train_2, df_eg_year], axis=1)
    df_val_2 = pd.concat([df_val_2, df_eg_year_val], axis=1)
    df_test_2 = pd.concat([df_test_2, df_eg_year_test], axis=1)

    #for imputation
    # s = df_train_2.isnull().sum()
    # c_list = np.where(s >= max(s)*0.5)
    # df_train_2 = df_train_2.drop(list(s.index[c_list[0]]), axis=1)

    # df_val_2 = df_val_2.drop(list(s.index[c_list[0]]), axis=1)
    # df_test_2 = df_test_2.drop(list(s.index[c_list[0]]), axis=1)

    # X_train, y_train, X_val, y_val, X_test = f_data(df_train_2, df_val_2, df_test_2)
    return df_train_2, df_val_2, df_test_2

### dataiwg

In [4]:
!pip install datawig

     |████████████████████████████████| 81kB 2.3MB/s 
     |████████████████████████████████| 25.4MB 35.4MB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


#비정규화

In [131]:
df_train_2, df_val_2, df_test_2 = preprocessing()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (8,172,173,174,175,176,177,178,179) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [0]:
df = df_train_2.select_dtypes("number")
df_val = df_val_2.select_dtypes("number")

In [0]:
c_list=["bldng_cnt","bldng_ar", "ttl_ar", "lnd_ar", "ttl_grnd_flr", "ttl_dwn_flr"]

In [147]:
import datawig
from tqdm import tqdm

temp_list = list(normalized_df[c_list].columns)

list_imputed = []

df_test_index = df[c_list][df[c_list].isnull().sum(axis=1) != 0].index
df_train_index = df[c_list][df[c_list].isnull().sum(axis=1) == 0].index
df_train, df_test = df[c_list].loc[df_train_index], df[c_list].loc[df_test_index]

for i in tqdm(range(len(temp_list))):

    
    temp = temp_list[i]
    temp_list.remove(temp)
    
    #Initialize a SimpleImputer model
    imputer = datawig.SimpleImputer(
        input_columns=temp_list, # column(s) containing information about the column we want to impute
        output_column= temp, # the column we'd like to impute values for
        output_path = 'imputer_model' # stores model data and metrics
        )

    #Fit an imputer model on the train data
    imputer.fit(train_df=df_train, num_epochs=50)

    #Impute missing values and return original dataframe with predictions
    imputed = imputer.predict(df_test)
    list_imputed.append(imputed)
    temp_list.insert(i, temp)


  0%|          | 0/6 [00:00<?, ?it/s]2019-12-03 14:35:41,303 [INFO]  
========== start: fit model
2019-12-03 14:35:41,306 [WARNING]  Already bound, ignoring bind()
2019-12-03 14:35:43,246 [INFO]  Epoch[0] Batch [0-1360]	Speed: 11257.30 samples/sec	cross-entropy=14.677700	bldng_cnt-accuracy=0.000000
2019-12-03 14:35:45,115 [INFO]  Epoch[0] Train-cross-entropy=48.075901
2019-12-03 14:35:45,118 [INFO]  Epoch[0] Train-bldng_cnt-accuracy=0.000000
2019-12-03 14:35:45,122 [INFO]  Epoch[0] Time cost=3.812
2019-12-03 14:35:45,130 [INFO]  Saved checkpoint to "imputer_model/model-0000.params"
2019-12-03 14:35:45,347 [INFO]  Epoch[0] Validation-cross-entropy=18.080627
2019-12-03 14:35:45,349 [INFO]  Epoch[0] Validation-bldng_cnt-accuracy=0.000000
2019-12-03 14:35:47,275 [INFO]  Epoch[1] Batch [0-1360]	Speed: 11342.76 samples/sec	cross-entropy=7.601200	bldng_cnt-accuracy=0.000000
2019-12-03 14:35:49,190 [INFO]  Epoch[1] Train-cross-entropy=21.295625
2019-12-03 14:35:49,192 [INFO]  Epoch[1] Train-b

In [72]:
list_imputed

[       bldng_cnt  bldng_ar  ...  ttl_dwn_flr  bldng_cnt_imputed
 4              2      0.00  ...          NaN           2.491304
 15             2    551.84  ...          NaN           3.867692
 19             4    238.36  ...          NaN           3.068296
 22             1    102.02  ...          NaN           2.746891
 33             4      0.00  ...          NaN           2.491304
 ...          ...       ...  ...          ...                ...
 59124          5    195.00  ...          NaN           2.977403
 59131          3    218.66  ...          NaN           3.040283
 59135          8   1917.37  ...          NaN           6.756392
 59180          3    126.32  ...          NaN           2.798636
 59194          2    423.00  ...          NaN           3.557026
 
 [11006 rows x 7 columns],
        bldng_cnt  bldng_ar  ...  ttl_dwn_flr  bldng_ar_imputed
 4              2      0.00  ...          NaN        653.367170
 15             2    551.84  ...          NaN       1112.113445

In [115]:
np.round(list_imputed[-2].ttl_grnd_flr_imputed)

4        2.0
15       3.0
19       2.0
22       2.0
33       2.0
        ... 
59124    1.0
59131    2.0
59135    5.0
59180    2.0
59194    2.0
Name: ttl_grnd_flr_imputed, Length: 11006, dtype: float64

In [91]:
df[df.ttl_grnd_flr.isnull()].index

Int64Index([    4,    15,    19,    22,    33,    34,    37,    44,    63,
               72,
            ...
            59036, 59058, 59075, 59080, 59111, 59124, 59131, 59135, 59180,
            59194],
           dtype='int64', length=10210)

In [141]:
df.ttl_grnd_flr = df.ttl_grnd_flr.fillna(np.round(list_imputed[-2].ttl_grnd_flr_imputed))
df.ttl_grnd_flr

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


0          1.0
1          1.0
2          5.0
3          1.0
4          NaN
         ...  
59194      NaN
59195    171.0
59196      4.0
59197      2.0
59198      1.0
Name: ttl_grnd_flr, Length: 59199, dtype: float64

In [144]:
df["ttl_dwn_flr"] = df.ttl_dwn_flr.fillna(np.round(list_imputed[-1].ttl_dwn_flr_imputed))
df.ttl_dwn_flr

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0        0.0
1        0.0
2        0.0
3        0.0
4        NaN
        ... 
59194    NaN
59195    0.0
59196    0.0
59197    0.0
59198    0.0
Name: ttl_dwn_flr, Length: 59199, dtype: float64

In [146]:
df.ttl_dwn_flr.fillna(np.round(list_imputed[-1].ttl_dwn_flr_imputed))

0        0.0
1        0.0
2        0.0
3        0.0
4        NaN
        ... 
59194    NaN
59195    0.0
59196    0.0
59197    0.0
59198    0.0
Name: ttl_dwn_flr, Length: 59199, dtype: float64

In [125]:
import datawig
from tqdm import tqdm

temp_list = list(normalized_df[c_list].columns)

list_imputed = []

df_test_index = df_val[c_list][df_val[c_list].isnull().sum(axis=1) != 0].index
df_train_index = df_val[c_list][df_val[c_list].isnull().sum(axis=1) == 0].index
df_train, df_test = df_val[c_list].loc[df_train_index], df_val[c_list].loc[df_test_index]

for i in tqdm(range(len(temp_list))):

    
    temp = temp_list[i]
    temp_list.remove(temp)
    
    #Initialize a SimpleImputer model
    imputer = datawig.SimpleImputer(
        input_columns=temp_list, # column(s) containing information about the column we want to impute
        output_column= temp, # the column we'd like to impute values for
        output_path = 'imputer_model' # stores model data and metrics
        )

    #Fit an imputer model on the train data
    imputer.fit(train_df=df_train, num_epochs=50)

    #Impute missing values and return original dataframe with predictions
    imputed = imputer.predict(df_test)
    list_imputed.append(imputed)
    temp_list.insert(i, temp)


  0%|          | 0/6 [00:00<?, ?it/s]2019-12-03 14:19:09,754 [INFO]  
========== start: fit model
2019-12-03 14:19:09,757 [WARNING]  Already bound, ignoring bind()
2019-12-03 14:19:09,992 [INFO]  Epoch[0] Batch [0-168]	Speed: 11902.80 samples/sec	cross-entropy=8.036580	bldng_cnt-accuracy=0.000000
2019-12-03 14:19:10,221 [INFO]  Epoch[0] Train-cross-entropy=7.058444
2019-12-03 14:19:10,224 [INFO]  Epoch[0] Train-bldng_cnt-accuracy=0.000000
2019-12-03 14:19:10,227 [INFO]  Epoch[0] Time cost=0.465
2019-12-03 14:19:10,235 [INFO]  Saved checkpoint to "imputer_model/model-0000.params"
2019-12-03 14:19:10,268 [INFO]  Epoch[0] Validation-cross-entropy=51.166677
2019-12-03 14:19:10,270 [INFO]  Epoch[0] Validation-bldng_cnt-accuracy=0.000000
2019-12-03 14:19:10,520 [INFO]  Epoch[1] Batch [0-168]	Speed: 11036.44 samples/sec	cross-entropy=8.550370	bldng_cnt-accuracy=0.000000
2019-12-03 14:19:10,769 [INFO]  Epoch[1] Train-cross-entropy=6.034992
2019-12-03 14:19:10,774 [INFO]  Epoch[1] Train-bldng_

In [126]:
list_imputed

[      bldng_cnt  bldng_ar  ...  ttl_dwn_flr  bldng_cnt_imputed
 6             2    235.01  ...          NaN           5.145618
 7             1    103.38  ...          NaN           1.673839
 15            2    159.88  ...          NaN           1.762300
 21            1    392.00  ...          NaN           2.072198
 25            1    133.08  ...          NaN           1.749115
 ...         ...       ...  ...          ...                ...
 6862          2    282.76  ...          NaN           1.895067
 6872          1    985.96  ...          NaN           2.728798
 6875          1    142.06  ...          NaN           1.731391
 6880          1    183.59  ...          NaN           2.103341
 6893          1      0.00  ...          NaN           1.553982
 
 [925 rows x 7 columns],
       bldng_cnt  bldng_ar   ttl_ar  ...  ttl_grnd_flr  ttl_dwn_flr  bldng_ar_imputed
 6             2    235.01   223.68  ...           2.0          NaN       4170.346001
 7             1    103.38   368.

In [136]:
df_val.ttl_grnd_flr = df_val.ttl_grnd_flr.fillna(np.round(list_imputed[-2].ttl_grnd_flr_imputed))
df_val.ttl_grnd_flr

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


0       1.0
1       1.0
2       3.0
3       1.0
4       1.0
       ... 
6893   -0.0
6894    2.0
6895    3.0
6896    3.0
6897    7.0
Name: ttl_grnd_flr, Length: 6898, dtype: float64

In [137]:
df_val.ttl_dwn_flr = df_val.ttl_grnd_flr.fillna(np.round(list_imputed[-1].ttl_dwn_flr_imputed))
df_val.ttl_dwn_flr

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


0       1.0
1       1.0
2       3.0
3       1.0
4       1.0
       ... 
6893   -0.0
6894    2.0
6895    3.0
6896    3.0
6897    7.0
Name: ttl_dwn_flr, Length: 6898, dtype: float64

In [0]:
df = df[df <= df.quantile(0.999)]

In [13]:
normalized_df=(df-df.min())/(df.max()-df.min())
normalized_df

,fr_yn,bldng_cnt,bldng_ar,ttl_ar,lnd_ar,dt_of_athrztn,ttl_grnd_flr,ttl_dwn_flr,tmprtr,prcpttn,wnd_spd,wnd_drctn,hmdt,lw_13101010,lw_13101110,lw_13101210,lw_13101211,lw_13101310,lw_13101410,lw_13111010,lw_13111110,lw_13121010,lw_13121011,lw_13131010,lw_13131110,lw_13141010,lw_13141011,hm_cnt,fr_sttn_dstnc,bldng_ar_prc,fr_wthr_fclt_dstnc,fr_mn_cnt,cctv_dstnc,fr_wthr_fclt_in_100m,cctv_in_100m,tbc_rtl_str_dstnc,sft_emrgnc_bll_dstnc,ahsm_dstnc,no_tbc_zn_dstnc,bldng_cnt_in_50m,e_14,e_15,e_16,e_17,e_18,e_t,g_14,g_15,g_16,g_17,g_18,g_t
0,1.0,0.032258,0.001080,0.000216,0.000000,0.889785,0.003704,0.0,0.500000,NaN,0.113924,0.555556,0.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.161231,0.137884,NaN,0.010241,0.416,0.009541,0.0,0.0,0.105519,0.291808,0.211967,0.007939,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.032258,0.000720,0.000144,0.000000,NaN,0.003704,0.0,0.677419,NaN,0.063291,0.055556,0.74,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.009850,0.193760,0.025442,0.037653,0.000,0.069682,0.0,0.0,0.252943,0.204433,0.063073,0.697068,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.000000,0.009078,0.007840,0.002271,0.951613,0.018519,0.0,0.596774,NaN,0.253165,0.250000,0.21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.160502,0.101250,0.145458,0.011011,0.416,0.014056,0.0,0.0,0.015457,0.311437,0.233104,0.006495,0.250000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.016129,0.000761,0.000152,0.000000,0.779570,0.003704,0.0,0.699597,NaN,0.050633,0.000000,0.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.063678,0.005311,0.170026,0.122045,0.728,0.011159,0.0,0.0,0.024441,0.042584,0.007620,0.045828,0.196429,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.016129,0.000000,0.000000,0.000000,NaN,NaN,NaN,0.540323,NaN,0.075949,1.000000,0.89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.160434,0.146230,NaN,0.046431,0.416,0.058438,0.0,0.0,0.094972,0.297269,0.233778,0.063780,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59194,0.0,0.016129,0.006578,0.001318,0.004728,0.940860,NaN,NaN,0.510081,NaN,0.025316,0.000000,0.41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.196187,0.104134,NaN,0.068915,0.432,0.096346,0.0,0.0,0.200379,0.036134,0.125042,0.082454,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
59195,0.0,0.548387,0.086985,0.170816,0.000000,NaN,0.633333,0.0,0.582661,NaN,0.240506,0.055556,0.72,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.196760,0.034388,0.443692,0.006776,0.432,0.009200,0.0,0.0,0.146699,0.000719,0.024057,0.011818,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
59196,0.0,0.016129,0.037894,0.007691,0.010229,NaN,0.014815,0.0,0.552419,NaN,0.405063,0.638889,0.58,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.021741,0.178190,NaN,0.001925,0.432,0.082460,0.5,0.0,0.238324,0.061245,0.091324,0.071267,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
59197,0.0,0.016129,0.004404,0.000647,0.000000,0.965054,0.007407,0.0,0.651210,0.027027,0.177215,1.000000,0.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.028927,0.260259,NaN,0.039270,0.432,0.015078,0.0,0.0,0.077395,0.101213,0.066874,0.055300,0.000000,0.0,0.0,0.000019,0.000018,0.000022,0.000019,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
normalized_df*(df.max()-df.min()) + df.min()

,fr_yn,bldng_cnt,bldng_ar,ttl_ar,lnd_ar,dt_of_athrztn,ttl_grnd_flr,ttl_dwn_flr,tmprtr,prcpttn,wnd_spd,wnd_drctn,hmdt,lw_13101010,lw_13101110,lw_13101210,lw_13101211,lw_13101310,lw_13101410,lw_13111010,lw_13111110,lw_13121010,lw_13121011,lw_13131010,lw_13131110,lw_13141010,lw_13141011,hm_cnt,fr_sttn_dstnc,bldng_ar_prc,fr_wthr_fclt_dstnc,fr_mn_cnt,cctv_dstnc,fr_wthr_fclt_in_100m,cctv_in_100m,tbc_rtl_str_dstnc,sft_emrgnc_bll_dstnc,ahsm_dstnc,no_tbc_zn_dstnc,bldng_cnt_in_50m,e_14,e_15,e_16,e_17,e_18,e_t,g_14,g_15,g_16,g_17,g_18,g_t
0,1.0,3.0,69.420,69.4200,0.0,1977.0,1.0,0.0,283.85,NaN,0.9,200.0,96.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17360.0,4547.0,NaN,133.0,137.0,112.0,0.0,0.0,1891.0,29231.0,11322.0,88.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,3.0,46.290,46.2900,0.0,NaN,1.0,0.0,292.65,NaN,0.5,20.0,74.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1791.0,6388.0,122581.0,489.0,85.0,818.0,0.0,0.0,4533.0,20480.0,3369.0,7727.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,583.800,2516.7600,1446.0,2000.0,5.0,0.0,288.65,NaN,2.0,90.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17285.0,3340.0,618105.0,143.0,137.0,165.0,0.0,0.0,277.0,31197.0,12451.0,72.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,2.0,48.920,48.9200,0.0,1936.0,1.0,0.0,293.75,NaN,0.4,0.0,91.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7327.0,179.0,719542.0,1585.0,176.0,131.0,0.0,0.0,438.0,4270.0,407.0,508.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,2.0,0.000,0.0000,0.0,NaN,NaN,NaN,285.85,NaN,0.6,360.0,89.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17278.0,4822.0,NaN,603.0,137.0,686.0,0.0,0.0,1702.0,29778.0,12487.0,707.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59194,0.0,2.0,423.000,423.0000,3010.0,1996.0,NaN,NaN,284.35,NaN,0.2,0.0,41.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20955.0,3435.0,NaN,895.0,139.0,1131.0,0.0,0.0,3591.0,3624.0,6679.0,914.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59195,0.0,35.0,5593.711,54835.0382,0.0,NaN,171.0,0.0,287.95,NaN,1.9,20.0,72.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21014.0,1137.0,1849455.0,88.0,139.0,108.0,0.0,0.0,2629.0,77.0,1285.0,131.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59196,0.0,2.0,2436.820,2468.8000,6512.0,NaN,4.0,0.0,286.45,NaN,3.2,230.0,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3014.0,5875.0,NaN,25.0,139.0,968.0,2.0,0.0,4271.0,6139.0,4878.0,790.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59197,0.0,2.0,283.200,207.7100,0.0,2005.0,2.0,0.0,291.35,1.0,1.4,360.0,95.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3753.0,8579.0,NaN,510.0,139.0,177.0,0.0,0.0,1387.0,10142.0,3572.0,613.0,0.0,0.0,0.0,3869.0,6348.0,8084.0,20008.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
# import datawig
# from tqdm import tqdm

# df_train, df_test = datawig.utils.random_split(normalized_df.select_dtypes("number"))
# list_imputed = []
# for i in tqdm(range(len(original_columns))):

#     temp = original_columns[i]
#     original_columns.remove(temp)

#     #Initialize a SimpleImputer model
#     imputer = datawig.SimpleImputer(
#         input_columns=original_columns, # column(s) containing information about the column we want to impute
#         output_column= temp, # the column we'd like to impute values for
#         output_path = 'imputer_model' # stores model data and metrics
#         )

#     #Fit an imputer model on the train data
#     imputer.fit(train_df=normalized_df, num_epochs=50)

#     #Impute missing values and return original dataframe with predictions
#     imputed = imputer.predict(df_test)
#     list_imputed.append(imputed)
#     original_columns.insert(i, temp)

In [17]:
normalized_df[c_list]

,bldng_ar,ttl_ar,lnd_ar
0,0.001080,0.000216,0.000000
1,0.000720,0.000144,0.000000
2,0.009078,0.007840,0.002271
3,0.000761,0.000152,0.000000
4,0.000000,0.000000,0.000000
...,...,...,...
59194,0.006578,0.001318,0.004728
59195,0.086985,0.170816,0.000000
59196,0.037894,0.007691,0.010229
59197,0.004404,0.000647,0.000000


In [61]:
[normalized_df[c_list].isnull().sum() == 0]

[bldng_ar    False
 ttl_ar      False
 lnd_ar      False
 dtype: bool]

# 정규화

In [64]:
import datawig
from tqdm import tqdm

temp_list = list(normalized_df[c_list].columns)

list_imputed = []

df_test_index = normalized_df[c_list][normalized_df[c_list].isnull().sum(axis=1) != 0].index
df_train_index = normalized_df[c_list][normalized_df[c_list].isnull().sum(axis=1) == 0].index
df_train, df_test = normalized_df[c_list].loc[df_train_index], normalized_df[c_list].loc[df_test_index]

for i in tqdm(range(len(temp_list))):

    
    temp = temp_list[i]
    temp_list.remove(temp)
    
    #Initialize a SimpleImputer model
    imputer = datawig.SimpleImputer(
        input_columns=temp_list, # column(s) containing information about the column we want to impute
        output_column= temp, # the column we'd like to impute values for
        output_path = 'imputer_model' # stores model data and metrics
        )

    #Fit an imputer model on the train data
    imputer.fit(train_df=df_train, num_epochs=50)

    #Impute missing values and return original dataframe with predictions
    imputed = imputer.predict(df_test)
    list_imputed.append(imputed)
    temp_list.insert(i, temp)

  0%|          | 0/3 [00:00<?, ?it/s]2019-12-03 08:51:42,628 [INFO]  
========== start: fit model
2019-12-03 08:51:42,630 [WARNING]  Already bound, ignoring bind()
2019-12-03 08:51:44,957 [INFO]  Epoch[0] Batch [0-1662]	Speed: 11477.48 samples/sec	cross-entropy=13.654160	bldng_ar-accuracy=0.000000
2019-12-03 08:51:47,434 [INFO]  Epoch[0] Train-cross-entropy=7.523230
2019-12-03 08:51:47,436 [INFO]  Epoch[0] Train-bldng_ar-accuracy=0.000000
2019-12-03 08:51:47,443 [INFO]  Epoch[0] Time cost=4.805
2019-12-03 08:51:47,448 [INFO]  Saved checkpoint to "imputer_model/model-0000.params"
2019-12-03 08:51:47,684 [INFO]  Epoch[0] Validation-cross-entropy=13.723686
2019-12-03 08:51:47,686 [INFO]  Epoch[0] Validation-bldng_ar-accuracy=0.000000
2019-12-03 08:51:50,146 [INFO]  Epoch[1] Batch [0-1662]	Speed: 10847.03 samples/sec	cross-entropy=10.320741	bldng_ar-accuracy=0.000000
2019-12-03 08:51:52,616 [INFO]  Epoch[1] Train-cross-entropy=6.695636
2019-12-03 08:51:52,623 [INFO]  Epoch[1] Train-bldng_a

KeyboardInterrupt: ignored

In [0]:
list_imputed